In [1]:
import suffix_tree
import pandas as pd
import os
import json
import pickle
from read_file import read_file
import node_op

In [2]:
find_encode = 1
brute = 1
drop_last = 1
"""
Segmentation method
- 0: Unique MT
- 1: Split by top repeaat
"""
seg_method = 1
ignore_1 = 1
MINIMAL_REPEAT = 5

In [3]:
def binary(string, length):
    while len(string) != length:
        string = '0' + string
    return string

# Input File

In [4]:
#current_path = os.path.join(os.path.expanduser("~"), "jupyter", "Fix_DCADE_Output")
#current_path = os.path.join(os.path.expanduser("~"), "Desktop", "DCADE", "TAM")
current_path = os.path.join(os.path.expanduser("~"), "Desktop", "test", "1")
#current_path = os.path.join(os.path.expanduser("~"), "Desktop", "test", "2")
input_file_path = os.path.join(current_path, "Output", "TXT", "TableA.txt")
print(input_file_path)

C:\Users\rick\Desktop\test\1\Output\TXT\TableA.txt


In [5]:
f = read_file(input_file_path)
content = f[0]
recb_start = f[1]
recb_end = f[2]
tag = f[3]
ids = f[4]
classes = f[5]
pathid = f[6]
parentid = f[7]
tecid = f[8]
cecid = f[9]
encoding = f[10]
col = f[11]
others = f[12]

In [6]:
tec_dict = {}
unique_mt = []
for node in range(len(col)):
    if col[node] == 'MT':
        if tecid[node] not in tec_dict.keys():
            tec_dict[tecid[node]] = [node]
        else:
            tec_dict[tecid[node]].append(node)
for key in tec_dict.keys():
    if len(tec_dict[key]) == 1:
        unique_mt += tec_dict[key]
print("Unique MT's index:\n", unique_mt)

Unique MT's index:
 [7, 175, 254, 295, 330, 340, 380, 420, 535, 560]


# Translate node in to unicode for suffix tree

Change `encode_option` to use different attribute to encode leafnodes.
<br><br>
Leafnodes will have same encoding if they have same attribute.

In [7]:
best = {'score': 0, 'option':'000000000'}
if find_encode == 1:
    check_dict = {}
    if brute == 1:
        candidate = []
        for i in range(7, 512):
            tmp = binary('{0:b}'.format(i), 9)
            if len(node_op.find_all_indexes(tmp, '1')) > 2:
                candidate.append(tmp)
    else:
        #candidate = ['000110100', '000101010', '000101001']
        with open('./good_encode.txt', 'rb') as f:
            candidate = pickle.load(f)
    for option in candidate:
        encode_option = option
        encode_col = [tag, ids, classes, pathid, parentid, tecid, cecid, encoding, col]
        """
        Node op
        ==================================================
        """
        node_encode = node_op.encode_node(encode_col, encode_option, len(pathid))
        whole_string = node_encode[0]
        node_dict = node_encode[1] # code -> node num
        index_dict = node_encode[2] # code -> first index
        """
        ===================================================
        """
        
        inv_node_dict = {v: k for k, v in node_dict.items()} # node num -> code
        inv_index_dict = {v: k for k, v in index_dict.items()} # index num -> code
        if seg_method == 0:
            segments = node_op.segment_mt(unique_mt, whole_string)
            record_seg = node_op.mt_record_seg(segments, ignore_1, index_dict, inv_node_dict, MINIMAL_REPEAT)
        if seg_method == 1:
            segments, record_seg = node_op.segment_top(whole_string, ignore_1, index_dict, inv_node_dict, MINIMAL_REPEAT)
        if len(record_seg) != 0 and str(record_seg) not in check_dict.keys():
            check_dict[str(record_seg)] = 1
            print(option)
            length = 0
            score = 0
            for s in record_seg:
                appear = []
                for c in s[1]:
                    if c not in appear:
                        appear.append(c)
                if len(appear) == 1: appear = []
                score += len(appear)
                length += len(s[1])
            print("Set count:", len(record_seg), "Length:", length, "Score:", score, "\n", record_seg)
            if score >= best['score']:
                best['score'] = score
                best['Length'] = length
                best['Set count'] = len(record_seg)
                best['option'] = option
print("Best:", best)

000001011
Set count: 1 Length: 2 Score: 2 
 [(0, '\x07\x08')]
000010011
Set count: 1 Length: 2 Score: 0 
 [(0, '\x00\x00')]
000100011
Set count: 1 Length: 2 Score: 0 
 [(0, '\x03\x03')]
000101001
Set count: 1 Length: 2 Score: 2 
 [(0, '\x08\t')]
000110001
Set count: 1 Length: 2 Score: 0 
 [(0, '\x05\x05')]
Best: {'score': 2, 'option': '000101001', 'Length': 2, 'Set count': 1}


In [8]:
if find_encode == 1:
    encode_option = best['option']
else:
    encode_option = '000110100'

encode_col = [tag, ids, classes, pathid, parentid, tecid, cecid, encoding, col]
"""
Node op
==================================================
"""
node_encode = node_op.encode_node(encode_col, encode_option, len(pathid))
whole_string = node_encode[0]
node_dict = node_encode[1] # code -> node num
index_dict = node_encode[2] # code -> first index
"""
===================================================
"""
print("Example: ")

for col_num in range(len(encode_col)):
    if encode_option[col_num] == '1':
        print(encode_col[col_num][node], end='')
inv_node_dict = {v: k for k, v in node_dict.items()} # node num -> code
inv_index_dict = {v: k for k, v in index_dict.items()} # index num -> code
#print("\nConvert to Unicode String:\n", whole_string)

Example: 
35-2MT

In [9]:
if seg_method == 0:
    segments = node_op.segment_mt(unique_mt, whole_string)

# Show maximal repeats

Ignore maximal repeats if length equal to 1.

In [10]:
if seg_method == 0:
    record_seg = node_op.mt_record_seg(segments, ignore_1, index_dict, inv_node_dict, MINIMAL_REPEAT)

In [11]:
if seg_method == 1:
    segments, record_seg = node_op.segment_top(whole_string, ignore_1, index_dict, inv_node_dict, MINIMAL_REPEAT)
    max_len = []
    all_seqs = []
    for seg_idx in range(len(record_seg)):
        max_len.append(0)
        seqs = []
        pos = node_op.find_all_indexes(segments[record_seg[seg_idx][0]], record_seg[seg_idx][1])
        for idx in range(len(pos)):
            if idx+1 != len(pos):
                seqs.append(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]])
                if len(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]]) > max_len[seg_idx]:
                    max_len[seg_idx] = len(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]])
            else:
                seqs.append(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx]+max_len[seg_idx]+4])
        all_seqs.append(seqs)
    #print(all_seqs)

# Whole String split by top repeats

In [12]:
if seg_method == 0:
    max_len = []
    all_seqs = []
    for seg_idx in range(len(record_seg)):
        max_len.append(0)
        seqs = []
        pos = node_op.find_all_indexes(segments[record_seg[seg_idx][0]], record_seg[seg_idx][1])
        for idx in range(len(pos)):
            if idx+1 != len(pos):
                seqs.append(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]])
                if len(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]]) > max_len[seg_idx]:
                    max_len[seg_idx] = len(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx+1]])
            else:
                seqs.append(segments[record_seg[seg_idx][0]][pos[idx]:pos[idx]+max_len[seg_idx]+4])
        all_seqs.append(seqs)
    print(all_seqs)

# MSA

In [13]:
import cstar
removed_whole_string = whole_string
for seg_idx in range(len(all_seqs)):
    json_result = []
    #print("First round MSA\n", "="*100)
    scores = [1, 0, -1] # matchScore, mismatchScore, gapScore
    if len(all_seqs[seg_idx][:-1]) == 1:
        msa = all_seqs[seg_idx][:-1]
    else:
        msa = cstar.CenterStar(scores, all_seqs[seg_idx][:-1]).msa()
    trans_dict = {}
    last_c = '-'
    end_idx = 0

    for i in range(len(msa)):
        if msa[i][-1] != '-' and last_c not in trans_dict.keys() and drop_last == 0:
            last_c = msa[i][-1]
            trans_dict[last_c] = msa[i].replace('-', '')
        if msa[i].replace('-', '') not in trans_dict.keys():
            trans_dict[msa[i].replace('-', '')] = msa[i]
        else: pass
    #for i in all_seqs[seg_idx][:-1]: print(i, "\n\t\t-> ", trans_dict[i])
    if drop_last == 0:
        #print('='*100, "\nSecond round MSA\n")
        msa_2 = cstar.CenterStar(scores, msa+[all_seqs[seg_idx][-1]]).msa()
        trans_dict_2 = {}

        for i in range(len(msa_2)):
            if msa_2[i].replace('-', '') not in trans_dict_2.keys():
                trans_dict_2[msa_2[i].replace('-', '')] = msa_2[i]
            else: pass
        for idx in range(len(trans_dict_2[trans_dict[last_c]])):
            if trans_dict_2[trans_dict[last_c]][idx] == last_c:
                end_idx = idx
        for i in all_seqs[seg_idx]:
            trans_dict_2[i] = trans_dict_2[i][:end_idx+1]
            tmp = trans_dict_2[i][:end_idx+1].replace('-', '')
            removed_whole_string = removed_whole_string.replace(tmp, '-'*len(tmp))
            #print(i, "\n\t\t-> ", trans_dict_2[i])

        json_schema = [{} for i in range(len(trans_dict_2[list(trans_dict_2.keys())[0]]))]
        schema_check = [0 for i in range(len(trans_dict_2[list(trans_dict_2.keys())[0]]))]
    else:
        json_schema = [{} for i in range(len(trans_dict[list(trans_dict.keys())[0]]))]
        schema_check = [0 for i in range(len(trans_dict[list(trans_dict.keys())[0]]))]

    with open(os.path.join(current_path, "Output", "Set-" + str(seg_idx) + ".txt"), 'w', encoding='utf-8') as file:
        for page in range(len(others)):
            json_page = []
            output_dict = {} # Record which pattern is used
            if drop_last == 1:
                length = len(all_seqs[seg_idx]) - 1
            else:
                length = len(all_seqs[seg_idx])
            for s in range(length):
                json_set = []
                file.write(str(page) + "-" + str(seg_idx) + "-" + str(s) + "\t")
                tmp = node_op.find_all_indexes(whole_string, all_seqs[seg_idx][s])
                if all_seqs[seg_idx][s] not in output_dict.keys():
                    output_dict[all_seqs[seg_idx][s]] = 0
                else:
                    output_dict[all_seqs[seg_idx][s]] += 1
                #print("start:", tmp[output_dict[seqs[s]]], others[page][tmp[output_dict[seqs[s]]]])
                idx = 0
                if drop_last == 0:
                    for c in range(len(trans_dict_2[all_seqs[seg_idx][s]])):
                        if trans_dict_2[all_seqs[seg_idx][s]][c] == '-':
                            file.write('\t')
                            json_set.append('')
                        else:
                            file.write(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")] + "\t")
                            json_set.append(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")])
                            if schema_check[c] == 0:
                                schema_check[c] = 1
                                json_schema[c]["PathId"] = pathid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["ParentId"] = parentid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx].split(':')[0].replace('\"', '')
                                if encoding[tmp[output_dict[all_seqs[seg_idx][s]]]+idx] == ' ':
                                    json_schema[c]["Encoding"] = ''
                                else:
                                    json_schema[c]["Encoding"] = int(encoding[tmp[output_dict[all_seqs[seg_idx][s]]]+idx])
                                json_schema[c]["CECId"] = cecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["TECId"] = tecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["ColType"] = col[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                            idx += 1
                else:
                    for c in range(len(trans_dict[all_seqs[seg_idx][s]])):
                        if trans_dict[all_seqs[seg_idx][s]][c] == '-':
                            file.write('\t')
                            json_set.append('')
                        else:
                            file.write(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")] + "\t")
                            json_set.append(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")])
                            if schema_check[c] == 0:
                                schema_check[c] = 1
                                json_schema[c]["PathId"] = pathid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["ParentId"] = parentid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx].split(':')[0].replace('\"', '')
                                if encoding[tmp[output_dict[all_seqs[seg_idx][s]]]+idx] == ' ':
                                    json_schema[c]["Encoding"] = ''
                                else:
                                    json_schema[c]["Encoding"] = int(encoding[tmp[output_dict[all_seqs[seg_idx][s]]]+idx])
                                json_schema[c]["CECId"] = cecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["TECId"] = tecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["ColType"] = col[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                            idx += 1
                json_page.append(json_set)
                file.write('\n')
            json_result.append(json_page)
    with open(os.path.join(current_path, "Output", "Set-" + str(seg_idx) + "_schema.json"), 'w') as json_file:
        json.dump(json_result, json_file)
    with open(os.path.join(current_path, "Output", "Set-" + str(seg_idx) + "_schema.json"), 'w') as json_file:
        json.dump(json_schema, json_file)

# Modified TableA Output

In [14]:
json_table = []
json_schema = []
for page in range(len(others)):
    json_page = []
    set_count = 0
    check = False
    for node in range(len(removed_whole_string)):
        schema_dict = {}
        if removed_whole_string[node] == '-':
            if check == False:
                check = True
                set_count += 1
                json_page.append(str(set_count) + '-' + str(page))
                schema_dict["PathId"] = ""
                schema_dict["ParentId"] = ""
                schema_dict["Encoding"] = -1
                schema_dict["CECId"] = ""
                schema_dict["TECId"] = ""
                schema_dict["ColType"] = "MR"
                json_schema.append(schema_dict)
            else: pass
        else:
            if check == True:
                check = False
            json_page.append(others[page][node][:others[page][node].find(" ::")])
            schema_dict["PathId"] = pathid[node]
            schema_dict["ParentId"] = parentid[node].split(':')[0].replace('\"', '')
            if encoding[node] != ' ':
                schema_dict["Encoding"] = int(encoding[node])
            else:
                schema_dict["Encoding"] = ''
            schema_dict["CECId"] = cecid[node]
            schema_dict["TECId"] = tecid[node]
            schema_dict["ColType"] = col[node]
            json_schema.append(schema_dict)
    json_table.append(json_page)
with open(os.path.join(current_path, "Output", "TableA.json"), 'w') as json_file:
    json.dump(json_table, json_file)
with open(os.path.join(current_path, "Output", "SchemaTableA.json"), 'w') as json_file:
    json.dump(json_schema, json_file)

# ===============================

In [15]:
print(len(all_seqs), "Set\nBEST: ", best)

1 Set
BEST:  {'score': 2, 'option': '000101001', 'Length': 2, 'Set count': 1}


In [16]:
with open('./good_encode.txt', 'rb') as f:
    candidate = pickle.load(f)
if best['option'] not in candidate:
    with open('./good_encode.txt', 'wb') as f:
        candidate.append(best['option'])
        print("Append:", best['option'])
        pickle.dump(candidate, f)

In [17]:
candidate

['000110100', '000101010', '000101001']